## Sentiment Analysis of Amazon Reviews on Musical Instruments using Machine Learning

1. Dataset: https://www.kaggle.com/eswarchandt/amazon-music-reviews
2. Problem statement: Given review text determine the polarity: Positive or Negative
3. Type of problem: Classification, Supervised
4. Data type: Review text and other parameters stored in csv file
5. Performance Measures: Accuracy, Precision, Recall, Confusion Matrix
6. Feature Importance: Not required
7. Interpretability: Why the review is classified as positive or negative

### Classification Algorithms:
1. K-Nearest Neighbor
2. Logistic Regression (one-vs-rest)
3. SVM Classifier
4. Decision Tree
5. Random Forest
6. XGBoost
7. Naive Bays

### Libraries required
1. Pandas
2. Numpy
3. Matplotlib and seaborn
4. nltk

### Dataset descrition

#### Content
This file has reviewer ID , User ID, Reviewer Name, Reviewer text, helpful, Summary(obtained from Reviewer text),Overall Rating on a scale 5, Review time
Description of columns in the file:

1. reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
2. asin - ID of the product, e.g. 0000013714
3. reviewerName - name of the reviewer
4. helpful - helpfulness rating of the review, e.g. 2/3
5. reviewText - text of the review
6. overall - rating of the product
7. summary - summary of the review
8. unixReviewTime - time of the review (unix time)
9. reviewTime - time of the review (raw) 

#### Important Features
1. reviewerID - Its unique for every customer hence can be removed
2. asin - ID of the product - Its unique for every customer hence can be removed
3. reviewerName - Does not impact on final sentiment of the review hence can be removed
4. helpful - Reveiw helpfulness may impact the polarity of the review text hence keep it by following modifications
   percentage of helpfulness = (number of customers found it helful) / (total number of customers found it helpful or not helpful)
5. reviewText: Most important text from which polarity is decided
6. summary: Short summary of the review text hence keep it
7. unixReviewTime: As this is time based data this feature will be helful for data partition into text/train/cv. But this feature is not going to help in deciding of polarity of review text
8. reviewTime: Removed as unixReviewTime is considered

### Import Libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import random
from random import randint
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

### Data Preprocessing

In [26]:
rawData = pd.read_csv("Musical_instruments_reviews.csv")

In [27]:
print("Data size shape",rawData.shape)
rawData.head()

('Data size shape', (10261, 9))


reviewerID        asin  \
0  A2IBPI20UZIR0U  1384719342   
1  A14VAT5EAX3D9S  1384719342   
2  A195EZSQDW3E21  1384719342   
3  A2C00NNG1ZQQG2  1384719342   
4   A94QU4C90B1AX  1384719342   

                                       reviewerName   helpful  \
0  cassandra tu "Yeah, well, that's just like, u...    [0, 0]   
1                                              Jake  [13, 14]   
2                     Rick Bennette "Rick Bennette"    [1, 1]   
3                         RustyBill "Sunday Rocker"    [0, 0]   
4                                     SEAN MASLANKA    [0, 0]   

                                          reviewText  overall  \
0  Not much to write about here, but it does exac...      5.0   
1  The product does exactly as it should and is q...      5.0   
2  The primary job of this device is to block the...      5.0   
3  Nice windscreen protects my MXL mic and preven...      5.0   
4  This pop filter is great. It looks and perform...      5.0   

                                 summary  unixReviewTime   reviewTime  
0                                   good      1393545600  02 28, 2014  
1                                   Jake      1363392000  03 16, 2013  
2                   It Does The Job Well      1377648000  08 28, 2013  
3          GOOD WINDSCREEN FOR THE MONEY      1392336000  02 14, 2014  
4  No more pops when I record my vocals.      1392940800  02 21, 2014

In [28]:
rawData.sort_values('unixReviewTime',ascending=True).reset_index() 

index      reviewerID        asin  \
0       4420   AV8MDYLHHTUOY  B000CD3QY2   
1       7413  A33H0WC9MI8OVW  B002Q0WT6U   
2        954  A33H0WC9MI8OVW  B0002D0COE   
3       5581  A3SMT15X2QVUR8  B000SZVYLQ   
4       1560  A3SMT15X2QVUR8  B0002E2EOE   
5       2004  A3SMT15X2QVUR8  B0002F73YY   
6       2590  A3SMT15X2QVUR8  B0002GXRF2   
7       3721  A1MI9FDCNB3CMR  B0006OHVK2   
8       1941   A1RPTVW5VEOSI  B0002F4MKC   
9       2632  A2PD27UKAD3Q00  B0002GXZK4   
10      3364  A12ABV9NU02O29  B0002MSQVQ   
11      5742  A27UAE2PME5TRV  B000VJ2VCK   
12       196  A1MI9FDCNB3CMR  B0000AQRST   
13      2556  A1MI9FDCNB3CMR  B0002GXF8Q   
14       194  A2KI91IR3RA7D0  B0000AQRST   
15       208  A2KI91IR3RA7D0  B0000AQRSU   
16      3881  A2G2DXPFZKG48W  B000978D58   
17      1426  A3SDJ70H00OQCJ  B0002E1NNM   
18      4254  A30J7WQV0ZNRXG  B000BBRZ3S   
19      4256   AU3GYRAKBUAEU  B000BBRZ3S   
20      3776  A30J7WQV0ZNRXG  B0007GGUGA   
21      1545  A30J7WQV0ZNRXG  B0002E1O90   
22      1448  A30J7WQV0ZNRXG  B0002E1O2C   
23       261   A5ZS85C5V40S8  B0002CZR3G   
24      3946   A5ZS85C5V40S8  B0009DXEEM   
25      7405  A3SDJ70H00OQCJ  B002Q0WSO8   
26      3730    A6FIAB28IS79  B0006VMBHI   
27      4539  A33H0WC9MI8OVW  B000EEHMXA   
28      4194   AEMC16IPBZORW  B000B6DHAS   
29       460   AEMC16IPBZORW  B0002CZVI2   
...      ...             ...         ...   
10231   2233  A2T8JRVJRVNX8R  B0002GJ6FC   
10232   5936   AKP6IWJ24C1A5  B000XXBHA6   
10233   9766  A3BMYEA3J6RBVV  B007YWHIP2   
10234   4711   AKP6IWJ24C1A5  B000EEN9OG   
10235   4769  A3MONHA5DCPPLZ  B000H2BC4E   
10236  10202  A356RFKNIG043B  B00H02C9TG   
10237  10123  A2JQWSA0GPVSQN  B00BU9KCXK   
10238  10199  A14JBDSWKPKTZA  B00H02C9TG   
10239   2591   AA5TINW2RJ195  B0002GXV2Q   
10240  10070   AA5TINW2RJ195  B00BKXN9LS   
10241   7603   AA5TINW2RJ195  B0037M62AK   
10242    667   AA5TINW2RJ195  B0002D01K4   
10243     41   AA5TINW2RJ195  B000068NW5   
10244  10260  A2WA8TDCTGUADI  B00JBIVXGC   
10245  10048  A35QR07ZW63T65  B00BG6ZM0K   
10246  10246  A29B4PAIOL7HYG  B00J4TBMVO   
10247  10113  A3KOWHTH4J4ZIK  B00BU9KCXK   
10248   7866  A3NHUK1ISMZCFZ  B003CLIPHO   
10249   5577  A3NHUK1ISMZCFZ  B000SZVYLQ   
10250   9058  A3NHUK1ISMZCFZ  B005CX4GLE   
10251   5165  A3NHUK1ISMZCFZ  B000NJ2TIE   
10252   3962  A3NHUK1ISMZCFZ  B0009EU01G   
10253   1001  A3NHUK1ISMZCFZ  B0002D0DWK   
10254  10256  A14B2YH83ZXMPP  B00JBIVXGC   
10255   6937  A3NHUK1ISMZCFZ  B001REJX1U   
10256  10119  A1P9JI4TR933JX  B00BU9KCXK   
10257  10215  A27L5L6I7OSV5B  B00HFRXACG   
10258  10253  A2VRAT69JDAD3W  B00JBIVXGC   
10259  10054  A3DJZHSAW4ANH0  B00BHK2UHI   
10260  10258   AWCJ12KBO5VII  B00JBIVXGC   

                                           reviewerName     helpful  \
0                              Amazon Customer "eyegor"    [18, 19]   
1                                             Clare Chu    [12, 13]   
2                                             Clare Chu     [9, 11]   
3      Victoria Tarrani "writer, editor, artist, des...    [63, 63]   
4      Victoria Tarrani "writer, editor, artist, des...    [58, 59]   
5      Victoria Tarrani "writer, editor, artist, des...    [21, 22]   
6      Victoria Tarrani "writer, editor, artist, des...    [22, 22]   
7                         Jorge Barbarosa "the_bassist"    [12, 13]   
8                                    Michael J. Edelman      [8, 9]   
9                  Wilhelmina Zeitgeist "coolartsybabe"  [156, 160]   
10                                             C. Longo      [1, 1]   
11                                                  kwb      [3, 3]   
12                        Jorge Barbarosa "the_bassist"    [15, 18]   
13                        Jorge Barbarosa "the_bassist"    [11, 24]   
14                                      David L. Vasser    [73, 76]   
15                                      David L. Vasser  [246, 259]   
16                                          Audio Maven    [52, 57]   


In [29]:
rawData.drop(['asin','reviewerName', 'reviewTime','reviewerID','helpful'], inplace = True, axis = 1)

In [39]:
rawData.head()

reviewText  overall  \
0  Not much to write about here, but it does exac...      5.0   
1  The product does exactly as it should and is q...      5.0   
2  The primary job of this device is to block the...      5.0   
3  Nice windscreen protects my MXL mic and preven...      5.0   
4  This pop filter is great. It looks and perform...      5.0   

                                 summary  unixReviewTime  
0                                   good      1393545600  
1                                   Jake      1363392000  
2                   It Does The Job Well      1377648000  
3          GOOD WINDSCREEN FOR THE MONEY      1392336000  
4  No more pops when I record my vocals.      1392940800

In [48]:
reviewTextNsummary = rawData[['reviewText','summary']]
ratings = rawData[['overall']]

In [49]:
reviewTextNsummary.head()
ratings.head()

overall
0      5.0
1      5.0
2      5.0
3      5.0
4      5.0

In [54]:
def newRatings(vals):
    if vals > 3.0:
        return 1.0
    else:
        return 0.0

In [55]:
ratings = ratings['overall'].apply(newRatings)

### Text -Data Processing 

#### Data Filtering
1. Convert all sentences into lower case
2. Remove special symbols
3. Remove urls
4. Remove digits
5. Convert i've, can't into i have can not etc.

#### NLP Processing
1. Tokenize sentences
2. Apply lemmatization
3. Convert text words into numerical vectors using
    1. Bag of Words
    2. TFIDF
    3. Word2Vec
    4. Average word2vec